# Simple iteration for systems of linear equations

First, generate a random diagonally dominant matrix, for testing.

In [228]:
import numpy as np
rndm = np.random.RandomState(1234)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

# I.  Jacobi iteration

Given

$$
A x = b
$$

separate the diagonal part $D$,

$$ A = D + (A - D) $$

and write

$$
x = D^{-1} (D - A) x + D^{-1} b\;.
$$

Then iterate

$$
x_{n + 1} = B x_{n} + c\;,
$$

where 

$$
B = D^{-1} (A - D) \qquad \text{and} \qquad c = D^{-1} b
$$


Let's construct the matrix and the r.h.s. for the Jacobi iteration

In [229]:
diag_1d = np.diag(A)

B = -A.copy()
np.fill_diagonal(B, 0)

D = np.diag(diag_1d)
invD = np.diag(1./diag_1d)
BB = invD @ B 
c = invD @ b

In [230]:
# sanity checks
from numpy.testing import assert_allclose

assert_allclose(-B + D, A)


# xx is a "ground truth" solution, compute it using a direct method
xx = np.linalg.solve(A, b)

np.testing.assert_allclose(A@xx, b)
np.testing.assert_allclose(D@xx, B@xx + b)
np.testing.assert_allclose(xx, BB@xx + c)

Check that $\| B\| \leqslant 1$:

In [231]:
np.linalg.norm(BB)

0.36436161983015336

### Do the Jacobi iteration

In [232]:
n_iter = 50

x0 = np.ones(n)
x = x0
for _ in range(n_iter):
    x = BB @ x + c

In [233]:
# Check the result:

A @ x - b

array([ 0.00000000e+00,  2.22044605e-16,  0.00000000e+00, -1.11022302e-16,
        0.00000000e+00,  0.00000000e+00, -2.08166817e-17,  0.00000000e+00,
        0.00000000e+00,  2.22044605e-16])

### Task I.1

Collect the proof-of-concept above into a single function implementing the Jacobi iteration. This function should receive the r.h.s. matrix $A$, the l.h.s. vector `b`, and the number of iterations to perform.


The matrix $A$ in the illustration above is strongly diagonally dominant, by construction. 
What happens if the diagonal matrix elements of $A$ are made smaller? Check the convergence of the Jacobi iteration, and check the value of the norm of $B$.

(20% of the total grade)


In [234]:
def Jacobi_iteration(A, b, n_iter):
    diag_1d = np.diag(A)

    B = -A.copy()
    np.fill_diagonal(B, 0)

    D = np.diag(diag_1d)
    invD = np.diag(1./diag_1d)
    BB = invD @ B 
    c = invD @ b
    
    x = np.ones(n)
    for _ in range(n_iter):
        x = BB @ x + c
        
    """Возвращает:
    ||B|| - норму матрицы B
     Δb   - разницу между A @ x и вектором b 
    """
    
    return np.linalg.norm(BB), A @ x-b


A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

print(Jacobi_iteration(A, b, 10))
print(Jacobi_iteration(A, b, 20))
print()

# Cделаем диагональные элементы в два раза меньше
# - для той же точности потребуется в два раза больше повторений
A = rndm.uniform(size=(n, n)) + np.diagflat([7]*n)
b = rndm.uniform(size=n)

print(Jacobi_iteration(A, b, 20))
print(Jacobi_iteration(A, b, 40))
print()

A = rndm.uniform(size=(n, n)) + np.diagflat([5]*n)
b = rndm.uniform(size=n)

print(Jacobi_iteration(A, b, 60))
print(Jacobi_iteration(A, b, 110))
print()

A = rndm.uniform(size=(n, n)) + np.diagflat([2]*n)
b = rndm.uniform(size=n)

# Если уменьшать диагональные момены, в определённый момент норма матрицы B станет больше 1
# 
print(Jacobi_iteration(A, b, 40))
print(Jacobi_iteration(A, b, 80))

(0.3679761958864619, array([1.16942618e-04, 8.46670912e-05, 8.84444342e-05, 9.06403170e-05,
       1.39709460e-04, 1.53519592e-04, 1.28330222e-04, 9.84880665e-05,
       9.82795627e-05, 1.31681085e-04]))
(0.3679761958864619, array([6.57111102e-10, 4.75752215e-10, 4.96977015e-10, 5.09316145e-10,
       7.85040144e-10, 8.62640515e-10, 7.21099291e-10, 5.53413426e-10,
       5.52241808e-10, 7.39928230e-10]))

(0.6907921005349461, array([1.34296197e-04, 1.73432087e-04, 9.49049918e-05, 1.73278057e-04,
       1.98050855e-04, 1.38879957e-04, 1.37158007e-04, 1.80803730e-04,
       1.49862491e-04, 1.30893980e-04]))
(0.6907921005349461, array([1.77567061e-09, 2.29312702e-09, 1.25483823e-09, 2.29109054e-09,
       2.61863764e-09, 1.83627727e-09, 1.81350968e-09, 2.39059533e-09,
       1.98148883e-09, 1.73068631e-09]))

(1.116708536930544, array([0.19156323, 0.17471991, 0.2106979 , 0.18306477, 0.17234526,
       0.17636312, 0.17758376, 0.17349296, 0.21549215, 0.20986031]))
(1.116708536930544, array(

# II. Seidel's iteration.

##### Task II.1

Implement the Seidel's iteration. 

Test it on a random matrix. Study the convergence of iterations, relate to the norm of the iteration matrix.

(30% of the total grade)

##### Seidel`s iteration description

Given

$$
A x = b
$$

separate the diagonal part D, upper trianglular and lower triangular parts U and L,

$$ A = L + U + D $$

and write

$$
x = - (D + L)^{-1} U x + (D + L)^{-1} b\;.
$$

Then iterate

$$
D x_{n + 1} + L x_{n + 1} + U x_{n} = b\;,
$$

Matrix of iterations is

$$
M = - (D + L)^{-1} U
$$

In [235]:
def Seidels_iteration(A, b, n_iter):
    n = A.shape[0]
    diag_1d = np.diag(A)

    D = np.diag(diag_1d)
    L = np.eye(n)
    U = np.eye(n)
    
    for i in range(n):
        for j in range(n-i):
            U[i,j+i] = A[i,j+i].copy()
    
    for i in range(n):
        for j in range(i):
            L[i,j] = A[i,j].copy()
    
    np.fill_diagonal(L, 0)
    np.fill_diagonal(U, 0)
    
    # Show matrices D, U and L
    # print((D*1e5)//1e2/1e3)
    # print((U*1e5)//1e2/1e3)
    # print((L*1e5)//1e2/1e3)
    
    x = np.ones(n)
    x1 = np.zeros(n)
    
    for i in range(n_iter):
        for k in range(n):
            x1[k] = (b[k] - np.dot(L[k,::], x1) - np.dot(U[k,::], x))/D[k,k]
        x = x1
        
        # Следим за последним элементом
        # print(x1[n-1])
        
    M = - np.linalg.inv(L+D) @ U

    """Возвращает:
    ||M|| - норму матрицы M
     Δb   - разницу между A @ x и вектором b 
    """
    return np.linalg.norm(M), A @ x - b

##### Series of tests

There is displayed observing of how diagonal elements value influences convergence of method.

In [236]:
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

print(Seidels_iteration(A, b, 10))
print(Seidels_iteration(A, b, 20))
print()

A = rndm.uniform(size=(n, n)) + np.diagflat([7]*n)
b = rndm.uniform(size=n)

print(Seidels_iteration(A, b, 10))
print(Seidels_iteration(A, b, 20))
print()

A = rndm.uniform(size=(n, n)) + np.diagflat([4]*n)
b = rndm.uniform(size=n)

print(Seidels_iteration(A, b, 10))
print(Seidels_iteration(A, b, 20))
print()

# Перестаёт сходиться
A = rndm.uniform(size=(n, n)) + np.diagflat(n)
b = rndm.uniform(size=n)

print(Seidels_iteration(A, b, 10))
print(Seidels_iteration(A, b, 20))

(0.2112162315549301, array([3.26338956e-12, 9.90985072e-13, 6.89159840e-12, 4.58244553e-12,
       4.97279995e-12, 1.77421966e-12, 3.36297656e-12, 7.41018358e-13,
       2.16604512e-13, 0.00000000e+00]))
(0.2112162315549301, array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  1.11022302e-16,
        0.00000000e+00, -6.93889390e-18,  0.00000000e+00, -5.55111512e-17,
        1.11022302e-16,  0.00000000e+00]))

(0.46508122481184316, array([ 1.52851166e-08,  3.75978430e-08,  5.17386667e-08,  2.64267643e-08,
        2.43330129e-08,  3.29431338e-09,  4.03871903e-09, -1.97347527e-09,
       -4.52838572e-10,  0.00000000e+00]))
(0.46508122481184316, array([ 1.11022302e-16,  2.77555756e-17,  0.00000000e+00,  0.00000000e+00,
       -1.11022302e-16, -5.55111512e-17,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00]))

(0.7137184917231388, array([-2.34393357e-07, -1.54717067e-07, -2.26365476e-07, -2.32619961e-07,
       -2.40343595e-07, -3.96763414e-08, -1.26255371e-

# III. Minimum residual scheme

### Task III.1

Implement the $\textit{minimum residual}$ scheme: an explicit non-stationary method, where at each step you select the iteration parameter $\tau_n$ to minimize the residual $\mathbf{r}_{n+1}$ given $\mathbf{r}_n$. Test it on a random matrix, study the convergence to the solution, in terms of the norm of the residual and the deviation from the ground truth solution (which you can obtain using a direct method). Study how the iteration parameter $\tau_n$ changes as iterations progress.

(50% of the grade)

In [ ]:
# ... ENTER YOUR CODE HERE ...